In [ ]:
pip install spotipy

In [60]:
import csv
import spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth


client_id = 'd061b0cb3ff347acb2968e6585f80104'
client_secret = '287bfbacf27f43f5bfdb325b389a7df5'


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
import pandas as pd

playlist_ids = [
    '37i9dQZF1DXcBWIGoYBM5M', 
    '37i9dQZEVXbLiRSasKsNU9', 
    '37i9dQZEVXbMDoHDwVN2tF', 
    '37i9dQZEVXbNG2KDcFcKOF',
    '4E0rXJSUur9qqbn0uZGtwE',
    '77kBUalZyju2tXEuPF36e8',
    '1W2Hms9hAHMBu71mEu0YuV',
    '6CxnmQPhTEpCoB6YtcvKUZ',
    '4y7mJB8JLrjhp2c6k3ADGC'
]

track_data = []

for playlist_id in playlist_ids:
    results = sp.playlist_tracks(playlist_id)
    for item in results['items']:
        track = item['track']
        track_data.append({
            'track_id': track['id'],
            'track_name': track['name'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'popularity': track['popularity']
        })

tracks_df = pd.DataFrame(track_data)
print(tracks_df.head())

In [62]:
audio_features = []
track_ids = tracks_df['track_id'].tolist()

for i in range(0, len(track_ids), 50): 
    features = sp.audio_features(track_ids[i:i+50])
    audio_features.extend(features)

audio_features_df = pd.DataFrame(audio_features)

final_dataset = pd.merge(tracks_df, audio_features_df, left_on='track_id', right_on='id', how='left')

final_dataset.drop(columns=['id'], inplace=True)


In [63]:
artist_data = []

artist_ids = tracks_df['artist_id'].unique().tolist()
for i in range(0, len(artist_ids), 50):
    artists = sp.artists(artist_ids[i:i+50])
    for artist in artists['artists']:
        artist_data.append({
            'artist_id': artist['id'],
            'artist_name': artist['name'],
            'genres': artist['genres']
        })

artists_df = pd.DataFrame(artist_data)
final_df = pd.merge(final_dataset, artists_df, on='artist_id', how='left')


In [64]:
final_df.to_csv('spotify_music_dataset.csv', index=False)

In [ ]:
print(final_df.head())
print(final_df.info())

In [66]:
final_df['genres'] = final_df['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [67]:
final_df = final_df.drop_duplicates()

duplicates_after = final_df.duplicated().sum()

In [68]:
columns_to_drop = ['type', 'uri', 'track_href', 'analysis_url', 'artist_name_y']
final_df = final_df.drop(columns=columns_to_drop)

In [ ]:
final_df.head()

In [70]:
tracks_df = final_df[['track_name']]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder_names = LabelEncoder()
final_df['track_name'] = label_encoder_names.fit_transform(final_df['track_name'])
tracks_df['track_name'] = label_encoder_names.fit_transform(final_df['track_name'])

label_encoder_artists = LabelEncoder()
final_df['artist_name_x'] = label_encoder_artists.fit_transform(final_df['artist_name_x'])

In [72]:
genre_counts = final_df['genres'].str.split(', ').explode().value_counts()

top_genres = genre_counts.head(15).index.tolist()

def filter_genres(genres):
    return [genre for genre in genres.split(', ') if genre in top_genres]

final_df['filtered_genres'] = final_df['genres'].apply(filter_genres)

genres_split = final_df['filtered_genres'].str.join(', ').str.get_dummies(sep=', ')

final_df = pd.concat([final_df, genres_split], axis=1)

final_df.drop(['genres', 'filtered_genres'], axis=1, inplace=True)


In [73]:
le = LabelEncoder()

final_df['track_id'] = le.fit_transform(final_df['track_id'])
final_df['artist_id'] = le.fit_transform(final_df['artist_id'])


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_columns = ['popularity', 'danceability', 'energy', 'key', 
                     'loudness', 'mode', 'speechiness', 
                     'acousticness', 'instrumentalness', 
                     'liveness', 'valence', 'tempo', 
                     'duration_ms', 'time_signature']

final_df[numerical_columns] = scaler.fit_transform(final_df[numerical_columns])
final_df.head()


In [ ]:
tracks_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


correlation_matrix = final_df.corr()
plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

In [ ]:
print(final_df.describe())

In [ ]:
features_to_plot = ['popularity', 'danceability', 'energy', 'valence', 'tempo']

for feature in features_to_plot:
    plt.figure(figsize=(8, 4))
    sns.histplot(final_df[feature], bins=30, kde=True)
    plt.title(f"Distribution of {feature}")
    plt.show()

In [75]:
final_df.to_csv('dataset.csv', index=False)
tracks_df.to_csv('tracks.csv', index=False)